# State Transition

This is an EKF implementation for sensor fusion of GPS and 9-axis IMU data for a 2D navigation problem. It assumes local coordinates X (northing) and y (easting), measured in meters from an arbitrary starting point. Heading is in degrees clockwise from north. Since GPS gives us course and speed while the IMU gives us heading, longitudinal acceleration, and yaw rate. This gives us the following state vector:

$$
\vec{x}_k = 
\begin{bmatrix}
x_k & y_k & \theta_k & v_k & \dot\theta_k & \dot v_k
\end{bmatrix}
$$

Where $k$ is the current time step, $x_k$ is the number of meters north of the origin, $y_k$ is the number of meters east, $\theta_k$ is heading in degrees clockwise of true north, $v_k$ is the velocity along the heading in meters per second, $\dot\theta_k$ is the yaw rate in clockwise degrees per second, and $\dot v_k$ is the acceleration along the heading in meters per second squared. The state transition functions from step $k-l$ to step $k$ are as follows:

\begin{align}
\hat{x}_k &=& x_{k-1} + \Delta{t}v_{k-1}\cos{\theta_{k-1}} + 0.5{\Delta{t}}^2\dot{v}_{k-1}\cos{\theta_{k-1}} \\
\hat{y}_k &=& y_{k-1} + \Delta{t}v_{k-1}\sin{\theta_{k-1}} + 0.5{\Delta{t}}^2\dot{v}_{k-1}\sin{\theta_{k-1}} \\
\hat{\theta}_k &=& \theta_{k-1} + \Delta{t}\dot\theta_{k-1} \\
\hat{v}_k &=& v_{k-1} + \Delta{t}\dot{v}_{k-1} \\
\hat{\dot\theta}_k &=& \dot\theta_{k-1} \\
\hat{\dot{v}}_k &=& \dot{v}_{k-1}
\end{align}

This of course is non-linear so we need to use an extended Kalman filter. 

# Sensor Inputs

We can ignore control inputs for the purposes of this EKF implementation, leaving only the $\bar{C}$ matrix to predict the value of the $\vec{z}_k$ sensor input vector from the estimated state $\hat{\vec{x}}_k$. We have seven inputs here:

$$
\vec{z}_k = 
\begin{bmatrix}
x_{gps_k} & y_{gps_k} & \theta_{gps_k} & \theta_{imu_k} & v_{gps_k} & \dot\theta_{imu_k} & \dot v_{imu_k}
\end{bmatrix}
$$

Where the subscript indicates the source of the sensor value being predicted. Note that this includes any necessary scaling and offsets, such as magnetic declination. The resulting $\bar{C}$ matrix is as follows:

$$
\bar{C} = 
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 \cr 
0 & 1 & 0 & 0 & 0 & 0 \cr 
0 & 0 & 1 & 0 & 0 & 0 \cr 
0 & 0 & 1 & 0 & 0 & 0 \cr 
0 & 0 & 0 & 1 & 0 & 0 \cr 
0 & 0 & 0 & 0 & 1 & 0 \cr 
0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}
$$

# Jacobian

The last piece to calculate is the Jacobian of the state transition function, $\bar{F}_k$. The value of the Jacobian is calculated at each time step according to the following equation:

$$
\bar{F}_k = 
\begin{bmatrix}
1 & 0 & -\Delta{t}v_k\sin{\theta_k} - 0.5{\Delta{t}}^2\dot{v}_k\sin{\theta_k} & \Delta{t}\cos{\theta_k} & 0 & 0.5{\Delta{t}}^2\dot{v}_k\cos{\theta_k} \cr
0 & 1 & \Delta{t}v_k\cos{\theta_k} + 0.5{\Delta{t}}^2\dot{v}_k\cos{\theta_k} & \Delta{t}\sin{\theta_k} & 0 & 0.5{\Delta{t}}^2\dot{v}_k\sin{\theta_k} \cr
0 & 0 & 1 & 0 & \Delta{t} & 0 \cr
0 & 0 & 0 & 1 & 0 & \Delta{t} \cr
0 & 0 & 0 & 0 & 1 & 0 \cr
0 & 0 & 0 & 0 & 0 & 1 
\end{bmatrix}
$$

# Noise Modeling

In order to make things simpler, we assume that the initial covariance estimate, $\bar{P}$ is an identity matrix as are the other noise matrices.